# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os
import numpy as np
import xarray as xr

import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

import cftime 
import datetime
from datetime import timedelta 


# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')

### Configure IMBIE comparison

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "GIS"# Change to "AIS" or "GIS"

# Set start and end dates
start_date = '2006-01-01'
end_date ='2014-01-01'

#Set density of ice
rho_ice = 918 # (kg/m^3)

#output file dirctory
output_path='/home/jovyan/CmCt/notebooks/IMBIE/'

# Set model model path,shapefile path and projection and IMBIE csv_file
if icesheet == "GIS":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    # nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_JPL_ISSM_asmb.nc'
    nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_ILTS_PIK_SICOPOLIS1_historical.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    #Set the observation data dir path
    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2021_Gt.csv'
    
elif icesheet== "AIS":
    projection = "EPSG:3031"  # Antarctica    
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    #Set the observation data dir path
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
    # obs_filename = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'
    
    ##Set the Region observation data dir path
    obs_east_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_east_antarctica_2022_Gt_partitioned_v0.csv'
    obs_west_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_west_antarctica_2022_Gt_partitioned_v0.csv'
    obs_peninsula_filename= '/home/jovyan/CmCt/data/IMBIE/imbie_antarctic_peninsula_2022_Gt_partitioned_v0.csv'

else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")


# Select  variable for mass balance comparision
mass_balance_column="Cumulative mass balance (Gt)"# "Cumulative dynamics mass balance anomaly (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"

In [3]:
# Check if  observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exists    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")


if icesheet== "AIS":   
    # Check if regional observation files exist 
    if not os.path.exists(obs_east_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
    if not os.path.exists(obs_west_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
    if not os.path.exists(obs_peninsula_filename):
        raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

## Load the model data

In [4]:
#Model data

nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_BGC_BISICLES_historical.nc'
gis_ds = xr.open_dataset(nc_filename)
lithk = gis_ds['lithk']
time_var = gis_ds['time']

### Check the selcted dates are within the range of model data

In [10]:
#Adjust the start and end date according to the time variable of model data
def adjust_for_calendar(calendar_type, date_dt, time_var):
    """
    Adjusts the date format to match the type of the time variable in the dataset.
    If the calendar is '360_day', adjust the day component accordingly.
    """
    # If the time_var is in numpy.datetime64, return the date as np.datetime64
    if isinstance(time_var.values[0], np.datetime64):
        return np.datetime64(date_dt)

    # If the time_var uses a cftime calendar, handle accordingly
    elif isinstance(time_var.values[0], cftime.datetime):
        if calendar_type == '360_day' and date_dt.day > 30:
            # In a 360-day calendar, each month has only 30 days.
            return cftime.datetime(date_dt.year, date_dt.month, 30, calendar=calendar_type)
        else:
            # For other calendars, use the original date.
            return cftime.datetime(date_dt.year, date_dt.month, date_dt.day, calendar=calendar_type)

    else:
        raise TypeError("Unsupported time format in the dataset.")
        

def convert_to_model_calendar(time_var, start_date, end_date):
    # Parse input dates to datetime objects
    start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    # Extract the calendar type used by the time variable
    try:
        calendar_type = time_var.to_index().calendar
        # print(calendar_type)
    except AttributeError:
        # Default to the 'standard' or 'gregorian' calendar if calendar attribute doesn't exist
        calendar_type = 'standard'
    
    # Convert the start_date and end_date to the correct calendar type
    start_date_cftime = adjust_for_calendar(calendar_type, start_date_dt,time_var)
    end_date_cftime = adjust_for_calendar(calendar_type, end_date_dt,time_var)
    
    # print(f"Start date in {calendar_type} calendar: {start_date_cftime}")
    # print(f"End date in {calendar_type} calendar: {end_date_cftime}")
    
    return start_date_cftime, end_date_cftime

In [11]:
# Get the minimum and maximum values directly from the time variable
min_time = time_var.values.min()
max_time = time_var.values.max()

fomatted_start_date, fomatted_end_date =convert_to_model_calendar(time_var, start_date, end_date)

# Check if the selected start and end dates are within the range
if min_time <= fomatted_start_date <= max_time and min_time <= fomatted_end_date <= max_time:
    print(f"The selected dates {start_date} and {end_date} are within the range of the model data.")
else:
    raise ValueError(f"Error: The selected dates {start_date} or {end_date} are out of range. Model data time range is from {min_time} to {max_time}.")


The selected dates 2006-01-01 and 2014-01-01 are within the range of the model data.


### Calculate  model mass balance for each basin and total mass balance for whole region

In [ ]:
# Interpolate lithk values at the start and end dates
lithk_start = lithk.interp(time=start_date).data.transpose().flatten()
lithk_end = lithk.interp(time=end_date).data.transpose().flatten()

# Calculate the difference
lithk_delta = lithk_end - lithk_start

# Replace NaN values with 0
lithk_delta[np.isnan(lithk_delta)] = 0


# Change Ice thickness unit from (m) to mass (kg) to gigatonnes(Gt)
# ice thickness*area* density of ice* 1e-12
#calculate area = x_resolution*y_resolution
x_coords = gis_ds['x'].values
y_coords = gis_ds['y'].values
x_resolution = abs(x_coords[1] - x_coords[0])
y_resolution = abs(y_coords[1] - y_coords[0])

lithk_delta = (lithk_delta * x_resolution*y_resolution)*rho_ice * 1e-12


# Create a list of Point geometries from coordinate grids
points = [Point(x, y) for x in x_coords for y in y_coords]

# Flatten lithk_delta to match the points list 
lithk_delta_flat = lithk_delta.flatten()

# Create DataFrame
lithk_df = pd.DataFrame({
    'geometry': points,
    'lithk_delta': lithk_delta_flat
})

# Convert DataFrame to GeoDataFrame
lithk_gdf = gpd.GeoDataFrame(lithk_df, geometry='geometry', crs=projection)

# Load basin shapefile 
basins_gdf = gpd.read_file(shape_filename)

# Perform spatial join
joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", predicate='intersects')

# Sum lithk_delta values by basin
if icesheet == "GIS":
     # Sum lithk_delta values by subregion column
    basin_mass_change_sums = joined_gdf.groupby('SUBREGION1')['lithk_delta'].sum()
elif icesheet == "AIS":
    # Sum lithk_delta values by subregion column
    basin_mass_change_sums = joined_gdf.groupby('Subregion')['lithk_delta'].sum()
    # Sum lithk_delta values by the 'Regions' column
    region_mass_change_sums = joined_gdf.groupby('Regions')['lithk_delta'].sum()
else:
    raise ValueError("Invalid iceshee value. Must be 'GIS' or 'AIS'.")

# Sum all of the basin mass change
model_total_mass_balance= basin_mass_change_sums.sum()

### IMBIE data date format conversion

In [ ]:
# Define a function to convert fractional years to a precise datetime format
def fractional_year_to_date(year):
    year_int = int(year)  # Extract the integer part (the full year)
    fraction = year - year_int  # Extract the fractional part
    
    # Start at the beginning of the year
    start_of_year = pd.Timestamp(f'{year_int}-01-01')
    
    # Determine if it's a leap year
    if pd.Timestamp(f'{year_int}-12-31').is_leap_year:
        total_days_in_year = 366
    else:
        total_days_in_year = 365
    
    # Convert the fractional part into the corresponding number of days
    fractional_days = fraction * total_days_in_year
    
    # Add the fractional days to the start of the year to get the correct date
    return start_of_year + timedelta(days=fractional_days)


# Group the data by year
def assign_month_order(group):
    # Get the month of the first entry for the year
    first_month = group['Date'].dt.month.iloc[0]
    
    # Create a month order starting from the first month and increasing by 1 for each subsequent entry
    group['Month_Order'] = range(first_month, first_month + len(group))
    return group

### Extract IMBIE mass balance data

In [ ]:
def sum_MassBalance(obs_filename,start_date,end_date):
    
    # Load the CSV file
    mass_balance_data = pd.read_csv(obs_filename)
    
    # Column names
    date_column = 'Year'
    
    # Ensure the 'Year' column is treated as float to capture the fractional year part
    mass_balance_data['Year'] = mass_balance_data['Year'].astype(float)
    
    # Apply the conversion function to the 'Year' column
    mass_balance_data['Date'] = mass_balance_data['Year'].apply(fractional_year_to_date)
  
    # Sort the data by 'Date' column to ensure it’s in increasing order of both year and fraction
    mass_balance_data = mass_balance_data.sort_values(by='Date')
      
    # Apply the function to each group of data (grouped by the year)
    mass_balance_data = mass_balance_data.groupby(mass_balance_data['Date'].dt.year).apply(assign_month_order)
    
    # Convert 'Year' column to year-month-01 format where month is 'Month_Order'
    mass_balance_data['Year'] = mass_balance_data.apply(lambda row: f"{row['Date'].year}-{str(row['Month_Order']).zfill(2)}-01", axis=1)
    
    # Reset the index to flatten the multi-index structure
    mass_balance_data = mass_balance_data.reset_index(drop=True)

    
    # Check if the column exists in the DataFrame
    if mass_balance_column not in mass_balance_data.columns:
        raise ValueError(f"Error: The column '{mass_balance_column}' does not exist in the CSV file.")

    
    # Filter the data for the end date
    end_data = mass_balance_data[mass_balance_data['Year'] == end_date]    
    if end_data.empty:
        raise ValueError(f"Error: No data available for the end date {end_date}.")
    mass_balance_end_value = end_data[mass_balance_column].iloc[-1]  # Last value before or at the end date

    
    # Filter the data for one date before the start date
    data_before_start_date = mass_balance_data[mass_balance_data[date_column] < start_date]
    if data_before_start_date.empty:
        raise ValueError(f"Error: No data available before the start date {start_date}.")
    mass_balance_start_value = data_before_start_date[mass_balance_column].iloc[-1]  # Last value before start date
    
    # Subtract the two values to get the total mass balance change
    IMBIE_total_mass_change_sum = mass_balance_end_value - mass_balance_start_value
    
    return IMBIE_total_mass_change_sum

### Calculate mass balance difference of IMBIE and model data

In [ ]:
#IMBIE total mass balance
IMBIE_total_mass_change_sum=sum_MassBalance(obs_filename,start_date,end_date)

# Calculate difference of IMBIE-model  mass change 
delta_masschange=IMBIE_total_mass_change_sum-model_total_mass_balance


In [ ]:
# Check if all required(regional) files are available for Antarctica
if icesheet == "AIS":
    print_regionalresult_check=[]
    if os.path.exists(obs_east_filename) and os.path.exists(obs_west_filename) and os.path.exists(obs_peninsula_filename):
        #Check
        print_regionalresult_check='YES' 
        
        # Calculate total mass for each region
        IMBIE_total_mass_change_sum_east = sum_MassBalance(obs_east_filename,start_date,end_date)
        IMBIE_total_mass_change_sum_west = sum_MassBalance(obs_west_filename,start_date,end_date)
        IMBIE_total_mass_change_sum_peninsula = sum_MassBalance(obs_peninsula_filename,start_date,end_date)

        # Calculate difference of IMBIE-model mass change for each region
        delta_masschange_east = IMBIE_total_mass_change_sum_east - region_mass_change_sums['East']
        delta_masschange_west = IMBIE_total_mass_change_sum_west - region_mass_change_sums['West']
        delta_masschange_peninsula = IMBIE_total_mass_change_sum_peninsula - region_mass_change_sums['Peninsula']

## Display result and write it to csv file

In [ ]:
# Initialize list to store rows of data for CSV
data_rows = []

# Add mass change comparison header as the first row with two columns
data_rows.append([f"Mass change comparison ({mass_balance_type})", f"{start_date} - {end_date}"])

# Add column headers for basin mass change
data_rows.append(['Basin', 'Model mass change (Gt)', 'IMBIE mass change (Gt)', 'Residual (Gt)'])


# Apply formatting to two decimal places for the model mass change
formatted_mass_change_sums = basin_mass_change_sums.apply(lambda x: f"{x:.2f}")

#Placeholders for 'IMBIE mass change' and 'Residual' columns
imbie_mass_change = '--'
residual_mass_change = '--'

print(f"\nMass change comparison ({mass_balance_type}): {start_date} - {end_date}")
# Define column headers with fixed width for alignment
print(f"{'Basin':<10} {'Model mass change (Gt)':<25} {'IMBIE mass change (Gt)':<25} {'Residual (Gt)':<25}")

# Loop through and print each basin's subregion mass change output with fixed-width formatting
for subregion, model_mass_change in formatted_mass_change_sums.items():
    print(f"{subregion:<10} {model_mass_change:<25} {imbie_mass_change:<25} {residual_mass_change:<25}")
    # Collect each basin's subregion mass change
    data_rows.append([subregion, model_mass_change, imbie_mass_change, residual_mass_change])

if icesheet == "AIS":
    if print_regionalresult_check == 'YES':
        # Remove 'Regions' and index name
        region_mass_change_sums.name = None
        region_mass_change_sums.index.name = None
        
        # Format the Series without displaying the 'dtype'
        formatted_region_mass_change = region_mass_change_sums.apply(lambda x: f"{x:.2f}")
        
        # Define regions, totals, and delta changes
        regions = ["East", "West", "Peninsula", "Islands"]
        IMBIE_totals = [IMBIE_total_mass_change_sum_east, IMBIE_total_mass_change_sum_west, 
                        IMBIE_total_mass_change_sum_peninsula]
        delta_changes = [delta_masschange_east, delta_masschange_west, 
                         delta_masschange_peninsula]
        
        # Loop through regions and print formatted output
        for region, total, delta in zip(regions, IMBIE_totals, delta_changes):
            mass_change = formatted_region_mass_change.get(region, "N/A")  # Get the mass change for the region
            print(f"{region:<10} {mass_change:<25} {total:<25.2f} {delta:<25.2f}")
            # Collect each region's mass change
            data_rows.append([region, mass_change, f"{total:.2f}" if total is not None else "N/A", 
                  f"{delta:.2f}" if delta is not None else "N/A"])

# Print total mass balance with formatted columns
print(f"{'Total':<10} {model_total_mass_balance.round(2):<25} {IMBIE_total_mass_change_sum:<25.2f} {delta_masschange:<25.2f}")


## Write the mass comaprision output results to csv files

In [ ]:
# Extract the base name of the nc file (without .nc extension)
nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')

# Create the CSV filename by combining the output path and the base nc filename with .csv extension
csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")


# Add the total mass balance row
data_rows.append(['Total', f"{model_total_mass_balance:.2f}", f"{IMBIE_total_mass_change_sum:.2f}", 
                  f"{delta_masschange:.2f}"])

# Convert the data rows into a pandas DataFrame
df = pd.DataFrame(data_rows)

# Write the DataFrame to a CSV file
print(f"Writing data to CSV file: {csv_filename}")
df.to_csv(csv_filename, index=False, header=False)